In [33]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/diffusion/')  #Root folder
sys.path.insert(0,'/content/drive/MyDrive/diffusion/utils/') # Libraries folder

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# EXPERIMENT PARAMETERS
in_timesteps = 8
out_timesteps = 12
in_features = 5
out_features = 2

meta_features = 2 # "case_id","object_id",

total_features = in_features + meta_features # because we need to keep object_id features and other identifiers
total_timesteps = in_timesteps + out_timesteps

test_dataset_path = '/content/drive/MyDrive/diffusion/data/DR_USA_Intersection_MA/_val.csv'

model_save_path = '/content/drive/MyDrive/diffusion/models/diffusion/'

# ADE function (single)

import math
def average_displacement_error(pred,ground):

    e = []

    for p,g in zip(pred,ground):
        e.append( math.sqrt (((p[0] - g[0]) ** 2) + ((p[1] - g[1]) ** 2)))

    return np.sum(e)/len(e),e[len(e)-1]   #ADE,FDE

# ADE function over BATCH (mean)

def calculate_error_batch(results, ground):

    ade = []
    fde = [] 

    for r, g in zip(results, ground):
        a,f = average_displacement_error(r,g)
        ade.append(a)
        fde.append(f)
    
    return np.sum(ade)/len(ade), np.sum(fde)/len(fde)


# Load test set
dataframe_test = pd.read_csv(test_dataset_path, sep=',')
dataframe_test["case_id"] = pd.to_numeric(dataframe_test["case_id"],downcast='integer')
dataframe_test["object_id"] = pd.to_numeric(dataframe_test["object_id"])

# load results
df_results_meta = pd.read_pickle(model_save_path+"df_results_meta.pkl")

# Concatenate results and ground truth
df = dataframe_test.copy()
df['x_hat'] = df_results_meta['x']
df['y_hat'] = df_results_meta['y']

# Select only position values
df_position = df[['object_id','x','y','x_hat','y_hat']]


# Calculate error for all trajectories

cars = df_position.groupby(['object_id'])

ade_list = list()
fde_list = list()

for ix,seq in cars:
    sub_seq = seq.to_numpy()
    prediction = sub_seq[8:,1:]
    ade,fde = average_displacement_error(prediction[:,2:],prediction[:,0:2])

    ade_list.append(ade)
    fde_list.append(fde)

avg_ade = round(sum(ade_list)/len(ade_list),3)
avg_fde = round(sum(fde_list)/len(fde_list),3)
    

print("avg_ade: "+ str(avg_ade))
print("avg_fde: "+str(avg_fde))




<ipython-input-33-199531869017>:79: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for ix,seq in cars:


avg_ade: 19.354
avg_fde: 20.939
